In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LinearRegression

In [3]:
df_gross = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")
df_gross['foreign_gross'] = df_gross['foreign_gross'].str.replace(',', '')
df_gross['foreign_gross'] = df_gross['foreign_gross'].astype(float)
df_gross.dropna(subset=['domestic_gross', 'foreign_gross'], inplace=True)
df_gross['total_gross']= df_gross['domestic_gross'] + df_gross['foreign_gross']

In [4]:
studio_group = df_gross.groupby('studio')
avg_total = studio_group['total_gross'].mean()
sorted_avg_total = avg_total.sort_values(ascending=False)
top10studios = pd.DataFrame(sorted_avg_total).head(10)

In [6]:
df_movies = pd.read_csv("./zippedData/tmdb.movies.csv.gz")
genre_mapping = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western"
}

import ast

def convert_str_list(s):
    try:
        return ast.literal_eval(s)
    except ValueError as e:
        print(f"Error for string: {s}\nError is: {e}")
        return s    # return the original string in case of error
def get_genre_names(genre_codes):
    return [genre_mapping.get(code, "Undefined") for code in genre_codes]

df_movies['genre_ids'] = df_movies['genre_ids'].apply(convert_str_list)
df_movies['genre_names'] = df_movies['genre_ids'].apply(get_genre_names)

df_movies['genre_names'] = df_movies['genre_names'].apply(lambda x: ', '.join(x))

In [7]:
df_movies['genre_names'] = df_movies['genre_names'].str.split(', ')
df_exploded = df_movies.explode('genre_names')
df_exploded['weighted_vote'] = df_exploded['vote_count'] * df_exploded['vote_average']
df_weighted = df_exploded.groupby('genre_names').agg({'weighted_vote': 'sum', 'vote_count': 'sum'})
df_weighted['average_rating'] = df_weighted['weighted_vote'] / df_weighted['vote_count']

In [8]:
df_weighted = df_weighted.sort_values(by='average_rating', ascending=False)
df_weighted.reset_index(inplace=True)
df_weighted

,genre_names,weighted_vote,vote_count,average_rating
0,History,1431103.3,195600,7.316479
1,War,1024805.5,141336,7.250846
2,Animation,3643815.3,506612,7.192517
3,Documentary,446405.3,62283,7.167370
4,Music,624432.3,87434,7.141756
5,Western,655407.2,92667,7.072714
6,Family,4588380.8,652585,7.031085
7,Drama,14485381.9,2064637,7.015946
8,Adventure,11288628.2,1645828,6.858936
9,Romance,4249937.6,621223,6.841243
